#### TensorFlow and Keras - Handwritten Digit Classification Using Neural Networks

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # نرمال‌سازی داده‌ها

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')

#### PyTorch - Same handwritten digit classification model with PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# بارگذاری و تبدیل داده‌ها
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        return torch.log_softmax(self.fc2(x), dim=1)

model = NeuralNet()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} completed")

correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        output = model(images)
        _, predicted = torch.max(output, 1)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {correct / len(test_set)}')


#### Scikit-Learn - Flower Classification Using SVM Model

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')


#### MXNet - Handwritten Digit Recognition with Neural Network

In [ ]:
import mxnet as mx
from mxnet import autograd, gluon, nd
from mxnet.gluon import nn, Trainer
from mxnet.gluon.data.vision import datasets, transforms

# بارگذاری و تبدیل داده‌ها
transformer = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0.13, 0.31)])
train_data = datasets.MNIST(train=True).transform_first(transformer)
test_data = datasets.MNIST(train=False).transform_first(transformer)

train_loader = mx.gluon.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = mx.gluon.data.DataLoader(test_data, batch_size=64, shuffle=False)

# ساختار مدل
net = nn.Sequential()
net.add(nn.Dense(128, activation='relu'))
net.add(nn.Dense(10))
net.initialize(mx.init.Xavier())

# آموزش مدل
trainer = Trainer(net.collect_params(), 'adam', {'learning_rate': 0.001})
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

for epoch in range(5):
    cumulative_loss = 0
    for data, label in train_loader:
        with autograd.record():
            output = net(data)
            loss = loss_fn(output, label)
        loss.backward()
        trainer.step(batch_size=64)
        cumulative_loss += loss.mean().asscalar()
    print(f'Epoch {epoch+1}, Loss: {cumulative_loss / len(train_loader)}')

# ارزیابی مدل
acc = mx.metric.Accuracy()
for data, label in test_loader:
    output = net(data)
    predictions = output.argmax(axis=1)
    acc.update(preds=predictions, labels=label)
print(f'Test Accuracy: {acc.get()[1]}')


#### XGBoost - Flower Classification with XGBoost

In [ ]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# بارگذاری مجموعه داده Iris
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

# ساخت و آموزش مدل XGBoost
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

# پیش‌بینی و ارزیابی
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
